In [1]:
import subprocess
import yaml
from time import sleep
import os
import numpy as np

In [2]:
def ticcmd(*args):
    return subprocess.check_output(['ticcmd'] + list(args))

def get_position():
    status = yaml.load(ticcmd('-s', '--full'))
    position = status['Current position']
    return position

def set_position(target_position):
    position = get_position()
    ticcmd('--resume', '--position', str(target_position))
    while position != target_position:
        position = get_position()
        print(position, end="\r")
        sleep(0.1)
    #sleep(0.5)
    #print(position)
    
def set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0):
    frequency_y2_adjusted = int(np.rint(co_3*target_frequency**3 + co_2*target_frequency**2 + co_1*target_frequency + co_0 - offset))
    set_position(frequency_y2_adjusted)
    
def reset_position(initial_position):
    set_position(initial_position - 200)
    set_position(initial_position)
    
def set_to_zero():
    zero_position = 0
    ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

In [22]:
initial_frequency = 2.818050
initial_position = -400
frequencies = [initial_frequency, 2.82      , 2.82067114, 2.82134228, 2.82201342, 2.82268456,
       2.8233557 , 2.82402685, 2.82469799, 2.82536913, 2.82604027,
       2.82671141, 2.82738255, 2.82805369, 2.82872483, 2.82939597,
       2.83006711, 2.83073826, 2.8314094 , 2.83208054, 2.83275168,
       2.83342282, 2.83409396, 2.8347651 , 2.83543624, 2.83610738,
       2.83677852, 2.83744966, 2.83812081, 2.83879195, 2.83946309,
       2.84013423, 2.84080537, 2.84147651, 2.84214765, 2.84281879,
       2.84348993, 2.84416107, 2.84483221, 2.84550336, 2.8461745 ,
       2.84684564, 2.84751678, 2.84818792, 2.84885906, 2.8495302 ,
       2.85020134, 2.85087248, 2.85154362, 2.85221477, 2.85288591,
       2.85355705, 2.85422819, 2.85489933, 2.85557047, 2.85624161,
       2.85691275, 2.85758389, 2.85825503, 2.85892617, 2.85959732,
       2.86026846, 2.8609396 , 2.86161074, 2.86228188, 2.86295302,
       2.86362416, 2.8642953 , 2.86496644, 2.86563758, 2.86630872,
       2.86697987, 2.86765101, 2.86832215, 2.86899329, 2.86966443,
       2.87033557, 2.87100671, 2.87167785, 2.87234899, 2.87302013,
       2.87369128, 2.87436242, 2.87503356, 2.8757047 , 2.87637584,
       2.87704698, 2.87771812, 2.87838926, 2.8790604 , 2.87973154,
       2.88040268, 2.88107383, 2.88174497, 2.88241611, 2.88308725,
       2.88375839, 2.88442953, 2.88510067, 2.88577181, 2.88644295,
       2.88711409, 2.88778523, 2.88845638, 2.88912752, 2.88979866,
       2.8904698 , 2.89114094, 2.89181208, 2.89248322, 2.89315436,
       2.8938255 , 2.89449664, 2.89516779, 2.89583893, 2.89651007,
       2.89718121, 2.89785235, 2.89852349, 2.89919463, 2.89986577,
       2.90053691, 2.90120805, 2.90187919, 2.90255034, 2.90322148,
       2.90389262, 2.90456376, 2.9052349 , 2.90590604, 2.90657718,
       2.90724832, 2.90791946, 2.9085906 , 2.90926174, 2.90993289,
       2.91060403, 2.91127517, 2.91194631, 2.91261745, 2.91328859,
       2.91395973, 2.91463087, 2.91530201, 2.91597315, 2.9166443 ,
       2.91731544, 2.91798658, 2.91865772, 2.91932886, 2.92]

co_3 = 69278.68
co_2 = -609102.93
co_1 = 1790504.24
co_0 = - 1762367.85

frequency_y1 = int(np.rint(co_3*initial_frequency**3 + co_2*initial_frequency**2 + co_1*initial_frequency + co_0))
offset = frequency_y1 - initial_position

counts = [0] * len(frequencies)
number_of_runs = 5
integration_time = 1
counter_frequency = 1/integration_time
sleep_time = 2

In [13]:
set_cavity_frequency(offset, 2.87, co_3, co_2, co_1, co_0)

In [19]:
reset_position(initial_position)

In [20]:
counterlogic.set_count_length(300)
counterlogic.set_count_frequency(counter_frequency)
counterlogic.startCount()
smiq.cw_on()

0

In [10]:
smiq.set_cw(2.88e9, -10)

(2880000000.0, -10.0, 'cw')

In [21]:
smiq.cw_on()

0

In [ ]:
for i in range(number_of_runs):
    for i in range(len(frequencies)):
        if i == 0:
            target_frequency = frequencies[i]
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]

        if i != 0:
            target_frequency = frequencies[i]
            set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0)
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]
            
    reset_position(initial_position)
            
print(counts)
print(frequencies)

In [ ]:
#Sets current position to 0
zero_position = 0
ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

#Get current count parameters
counterlogic.get_count_length()
counterlogic.get_count_frequency()

# Brute force way to control anything, pull gui levers.
counter._mw.start_counter_Action.trigger()

In [3]:
get_position()

-500

In [5]:
set_position(-400)

In [31]:
count = 4

In [32]:
count

4

In [49]:
target = get_position() + 100
count = count + 1

set_position(target)
print("The number is", count, "       ")

The number is 21        


In [21]:
smiq.cw_on()

0

In [14]:
set_to_zero()

In [ ]:
reset_position(-4800)

In [ ]:
smiq.off()


In [15]:
np.linspace(2.82, 2.92, 150)

array([2.82      , 2.82067114, 2.82134228, 2.82201342, 2.82268456,
       2.8233557 , 2.82402685, 2.82469799, 2.82536913, 2.82604027,
       2.82671141, 2.82738255, 2.82805369, 2.82872483, 2.82939597,
       2.83006711, 2.83073826, 2.8314094 , 2.83208054, 2.83275168,
       2.83342282, 2.83409396, 2.8347651 , 2.83543624, 2.83610738,
       2.83677852, 2.83744966, 2.83812081, 2.83879195, 2.83946309,
       2.84013423, 2.84080537, 2.84147651, 2.84214765, 2.84281879,
       2.84348993, 2.84416107, 2.84483221, 2.84550336, 2.8461745 ,
       2.84684564, 2.84751678, 2.84818792, 2.84885906, 2.8495302 ,
       2.85020134, 2.85087248, 2.85154362, 2.85221477, 2.85288591,
       2.85355705, 2.85422819, 2.85489933, 2.85557047, 2.85624161,
       2.85691275, 2.85758389, 2.85825503, 2.85892617, 2.85959732,
       2.86026846, 2.8609396 , 2.86161074, 2.86228188, 2.86295302,
       2.86362416, 2.8642953 , 2.86496644, 2.86563758, 2.86630872,
       2.86697987, 2.86765101, 2.86832215, 2.86899329, 2.86966

In [9]:
counterlogic.countdata[0,-1]

922329.0